In [ ]:
from cling import cling, bash

**Listing 8.1**

Caption: Implementation of the Taylor series of the natural logarithm using std::thread.

In [ ]:
%%cling
#include <algorithm>
#include <cmath>
#include <cstdlib>
#include <iostream>
#include <numeric>
#include <string>
#include <thread>
#include <vector>
//-----------------------------------------------------------------
// Get the number of iterations
size_t n = 6; // std::stoi(argv[1]);
// Get the amount of threads
size_t num_threads = 3; // std::stoi(argv[3]); 
// Compute the partition size for each thread
size_t partition_size = std::round(n / num_threads); 
// the value of x
double x = 0.1;

std::vector<double> parts(n);
std::iota(parts.begin(), parts.end(), 1);

using std::thread;

std::vector<thread> threads;
for (size_t i = 0; i < num_threads; i++) {  
  size_t begin = i * partition_size; 
  size_t end = (i + 1) * partition_size;
  if (i == num_threads - 1)
    end = n;  

  thread t([begin, end]() { 
    std::for_each(parts.begin() + begin, parts.begin() + end,
                  [](double &e) { 
                    e = std::pow(-1.0, e + 1) * std::pow(x, e) / e;
                  });
  });

  threads.push_back(std::move(t)); 
}

for (thread &t : threads) {
  t.join(); 
}
// Collect the partial results once all  threads finished
double result = std::accumulate(parts.begin(), parts.end(), 0.); 


**Listing 8.2**

Caption: Parallel implementation of the fractal set using std::thread.

In [ ]:
%%cling
#include <pbm.hpp>
#include <config.hpp>
#include <kernel.hpp>

#include <algorithm>
#include <cstdlib>
#include <iostream>
#include <numeric>
#include <pbm.hpp>
#include <thread>
//-----------------------------------------------------------------
PBM pbm = PBM(size_x, size_y);

std::vector<size_t> index_(size_x);
std::iota(index_.begin(), index_.end(), 0);

using std::thread;

std::vector<thread> threads; 
const size_t nthreads = 4; // Choose the number

// Calculate the size of each partition
const int delx = index_.size() / nthreads; 

for (size_t n = 0; n < nthreads; n++) {   
  // Split into equal chunks
  const int nlo = n * delx; 
  const int nhi = n + 1 == nthreads ? index_.size() : (n + 1) * delx; 

  thread t([n,nlo,nhi]() {
    std::for_each(index_.begin() + nlo, index_.begin() + nhi, [n,nlo,nhi](size_t i) { 
      complex c =
          complex(0, 4) * complex(i, 0) / complex(size_x, 0) - complex(0, 2);

      for (size_t j = 0; j < size_y; j++) {
        // Get the number of iterations
        int value = compute_pixel(c + 4.0 * j / size_y - 2.0);
        // Convert the value to RGB color space
        std::tuple<size_t, size_t, size_t> color = get_rgb(value);
        // Set the pixel color
        pbm(i, j) = make_color(std::get<0>(color),
                               std::get<1>(color),
                               std::get<2>(color));
      }
    });
  });
  threads.push_back(std::move(t)); 
}
for (thread &t : threads) {
  t.join();  
}
